In [ ]:
%cd /data/jupyter/root/ECNU/DataMining/L14-Recommender/

In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# engine = create_engine('mysql+pymysql://root:@Jiangzg59915431022.@localhost:3306/all_gzdata?charset=utf8')

In [4]:
# engine.connect()

In [137]:
sql = pd.read_csv("/var/lib/phpMyAdmin/save/all_gzdata.csv", chunksize = 10000)

# 统计

In [46]:
counts = [ i['fullURLId'].value_counts() for i in sql] #逐块统计

In [47]:
counts

[101003     4762
 1999001    2389
 107001     2143
 301001      177
 101002      166
 102002      163
 101001       51
 106001       37
 103003       26
 101009       18
 102001       15
 103002       15
 102005       10
 101008        8
 102003        6
 102008        4
 101007        3
 102009        2
 102006        2
 101006        1
 101005        1
 101004        1
 Name: fullURLId, dtype: int64, 101003     4832
 107001     2306
 1999001    2269
 301001      154
 102002      151
 101002      134
 101001       53
 106001       40
 102001       14
 103003       11
 102003        8
 101009        6
 103002        5
 102005        4
 101008        3
 101006        3
 102006        3
 102007        2
 101004        1
 102004        1
 Name: fullURLId, dtype: int64, 101003     4755
 107001     2314
 1999001    2104
 102002      239
 301001      184
 101002      163
 101001       64
 106001       52
 102001       33
 102003       24
 103003       23
 102005       12
 101009       11
 10

In [48]:
counts1 = pd.concat(counts).groupby(level=0).sum()
counts1

101001       5603
101002       7776
101003     396612
101004        125
101005         63
101006        107
101007        147
101008        378
101009        854
102001       2129
102002      12021
102003       1235
102004        361
102005        271
102006        184
102007        538
102008        404
102009        214
103002        189
103003       1526
106001       3957
107001     182900
301001      18430
1999001    201426
Name: fullURLId, dtype: int64

In [50]:
counts2 = counts1.reset_index() #重新设置index，将原来的index作为counts的一列。
counts2

,index,fullURLId
0,101001,5603
1,101002,7776
2,101003,396612
3,101004,125
4,101005,63
5,101006,107
6,101007,147
7,101008,378
8,101009,854
9,102001,2129


In [57]:
counts2.columns = ['index', 'num'] #重新设置列名，主要是第二列，默认为0
counts2

,index,num
0,101001,5603
1,101002,7776
2,101003,396612
3,101004,125
4,101005,63
5,101006,107
6,101007,147
7,101008,378
8,101009,854
9,102001,2129


In [64]:
counts2['type'] = counts2['index'].astype(str).str.extract('(\d{3})',expand=True) #提取前三个数字作为类别id
counts2

,index,num,type
0,101001,5603,101
1,101002,7776,101
2,101003,396612,101
3,101004,125,101
4,101005,63,101
5,101006,107,101
6,101007,147,101
7,101008,378,101
8,101009,854,101
9,102001,2129,102


In [67]:
counts_ = counts2[['type', 'num']].groupby('type').sum() #按类别合并
counts_.sort_values(by='num', ascending = False) #降序排列
counts_

,num
type,
101,411665
102,17357
103,1715
106,3957
107,182900
199,201426
301,18430


# 统计分析 -- 知识类型页面107001

In [78]:
#统计107类别的情况
def count107(i): #自定义统计函数
  j = i[['fullURL']][i['fullURLId'].astype(str).str.contains('107')].copy() #找出类别包含107的网址
  j['type'] = None #添加空列
  j['type'][j['fullURL'].astype(str).str.contains('info/.+?/')] = u'知识首页'
  j['type'][j['fullURL'].astype(str).str.contains('info/.+?/.+?')] = u'知识列表页'
  j['type'][j['fullURL'].astype(str).str.contains('/\d+?_*\d+?\.html')] = u'知识内容页'
  return j['type'].value_counts()

counts2 = [count107(i) for i in sql] #逐块统计
counts2

[知识内容页    1960
 知识列表页     111
 知识首页       72
 Name: type, dtype: int64, 知识内容页    2070
 知识首页      128
 知识列表页     108
 Name: type, dtype: int64, 知识内容页    2094
 知识列表页     128
 知识首页       92
 Name: type, dtype: int64, 知识内容页    2255
 知识列表页     115
 知识首页       56
 Name: type, dtype: int64, 知识内容页    2047
 知识列表页     134
 知识首页       95
 Name: type, dtype: int64, 知识内容页    1926
 知识首页      119
 知识列表页     119
 Name: type, dtype: int64, 知识内容页    2197
 知识列表页     125
 知识首页       90
 Name: type, dtype: int64, 知识内容页    2089
 知识列表页     105
 知识首页       79
 Name: type, dtype: int64, 知识内容页    2087
 知识列表页     106
 知识首页       64
 Name: type, dtype: int64, 知识内容页    2180
 知识首页      120
 知识列表页      99
 Name: type, dtype: int64, 知识内容页    2115
 知识列表页     114
 知识首页       60
 Name: type, dtype: int64, 知识内容页    2021
 知识列表页     102
 知识首页       54
 Name: type, dtype: int64, 知识内容页    2160
 知识列表页     117
 知识首页       68
 Name: type, dtype: int64, 知识内容页    1900
 知识列表页     159
 知识首页       76
 Name: type, dtype: int64, 知识内容页

In [75]:
counts3 = pd.concat(counts2).groupby(level=0).sum() #合并统计结果
counts3

知识内容页    160213
知识列表页      9437
知识首页       8801
Name: type, dtype: int64

In [81]:
# 统计点击次数
c = [ i['realIP'].value_counts() for i in sql] #逐块统计各个IP的出现次数
c

[2609113527    420
 3812410744    360
 3875285307    169
 3961223025    104
 1271225968    103
              ... 
 1701319694      1
 4100746609      1
 2143689742      1
 1889870201      1
 3365494384      1
 Name: realIP, Length: 4510, dtype: int64, 2609113527    361
 3812410744    317
 3394278670    138
 3841730875    131
 3875285307    128
              ... 
 424703758       1
 2595723790      1
 3877132413      1
 3327555598      1
 211332988       1
 Name: realIP, Length: 4524, dtype: int64, 2609113527    571
 3812410744    307
 3875285307    147
 1383154801    116
 1207702030     58
              ... 
 1642496631      1
 1792246135      1
 1560904311      1
 773067738       1
 1336630896      1
 Name: realIP, Length: 4553, dtype: int64, 2609113527    829
 3812410744    385
 3875285307    110
 213549687     103
 4256397582     76
              ... 
 1887446331      1
 1640006007      1
 2429949812      1
 4112146545      1
 3046720113      1
 Name: realIP, Length: 4200, dtype: in

In [82]:
counts4 = pd.concat(c).groupby(level=0).sum()#合并统计结果，level=0表示按index分组，然后求和
counts4

82033         2
95502         1
103182        1
116010        2
136206        1
             ..
4294809358    2
4294811150    1
4294852154    3
4294865422    2
4294917690    1
Name: realIP, Length: 230149, dtype: int64

In [86]:
counts5 = pd.DataFrame(counts4) #将Series转为DataFrame
counts5[1] = 1 #添加一列，值全为1
counts5

,realIP,1
82033,2,1
95502,1,1
103182,1,1
116010,2,1
136206,1,1
...,...,...
4294809358,2,1
4294811150,1,1
4294852154,3,1
4294865422,2,1


In [89]:
counts5.groupby('realIP').sum() # 统计各个‘不同点击次数’分别出现的频次 点x次的用户有多少个

,1
realIP,
1,132119
2,44175
3,17573
4,10156
5,5952
...,...
2000,1
2413,1
3306,1


In [107]:
c3_ = counts5.groupby('realIP').sum() / len(counts5) * 100 # len(counts5) 所有用户
c3_.head()

,1
realIP,
1,57.405854
2,19.194087
3,7.635488
4,4.412793
5,2.586151


In [120]:
#大约84.2%的用户点击不超过3次，占总浏览量的31%(不会求1~~~)

# 数据预处理

## 数据清洗

In [12]:
sql = pd.read_csv("/var/lib/phpMyAdmin/save/all_gzdata.csv", chunksize = 10000)
for i in sql:
  d = i[['realIP', 'fullURL']] #只要网址列
  d = d[d['fullURL'].str.contains('\.html')].copy() #只要含有.html的网址
  #保存到数据库的cleaned_gzdata表中（如果表不存在则自动创建）
  d.to_csv('data/cleaned_gzdata.csv', mode='a+', header=(False if not need_head else True))

NameError: name 'need_head' is not defined

In [23]:
sql_clear = pd.read_csv("data/cleaned_gzdata.csv", chunksize = 10000, names=['a', 'realIP', 'fullURL'])
for i in sql_clear:
    a = i
    break
a

,a,realIP,fullURL
0,0,2683657840,http://www.lawtime.cn/info/hunyin/hunyinfagui/...
1,1,973705742,http://www.lawtime.cn/ask/exp/17199.html
2,2,3104681075,http://www.lawtime.cn/ask/question_3893276.html
3,3,308351962,http://www.lawtime.cn/ask/question_5281741.html
4,4,2683657840,http://www.lawtime.cn/info/hunyin/hunyinfagui/...
...,...,...,...
9995,12269,1201390890,http://www.lawtime.cn/faguizt/117.html
9996,12270,2852324818,http://www.lawtime.cn/ask/question_10217550.html
9997,12271,2852324818,http://www.lawtime.cn/ask/question_10237869.html
9998,12272,3697296241,http://www.lawtime.cn/ask/question_3062766.html


## 数据变换

### 删去分页网址

In [24]:
for i in sql_clear: #逐块变换并去重
  d = i.copy()
  d['fullURL'] = d['fullURL'].str.replace('_\d{0,2}.html', '.html') #将下划线后面部分去掉，规范为标准网址
  d = d.drop_duplicates() #删除重复记录
  d.to_csv('data/changed_gzdata.csv', mode='a+', header=(False if not need_head else True))

In [5]:
sql_change = pd.read_csv("data/changed_gzdata.csv", names=['a', 'realIP', 'fullURL'])
sql_change

,a,realIP,fullURL
10000,12274,707080375,http://www.lawtime.cn/ask/question_2809224.html
10001,12275,4195674807,http://www.lawtime.cn/ask/question_1009369.html
10002,12276,2020773949,http://www.lawtime.cn/faguizt/23.html
10003,12277,2020773949,http://www.lawtime.cn/faguizt/23.html
10004,12278,1352628081,http://www.lawtime.cn/ask/question_9878938.html
...,...,...,...
673016,837441,3399783182,http://www.lawtime.cn/ask/question_10030566.html
673017,837442,2324433017,http://www.lawtime.cn/info/minshi/fagui/201401...
673018,837444,1375758705,http://www.lawtime.cn/info/shengchan/aqzs/2008...
673019,837445,2324433017,http://www.lawtime.cn/info/minshi/fagui/201401...


### 人工规则分类

In [49]:
sql_change = pd.read_csv("data/changed_gzdata.csv", names=['a', 'b', 'realIP', 'fullURL'])

In [50]:
d = sql_change[:1000]
d.head()

,a,b,realIP,fullURL
0,10000,12274,707080375,http://www.lawtime.cn/ask/question_2809224.html
1,10001,12275,4195674807,http://www.lawtime.cn/ask/question_1009369.html
2,10002,12276,2020773949,http://www.lawtime.cn/faguizt/23.html
3,10003,12277,2020773949,http://www.lawtime.cn/faguizt/23.html
4,10004,12278,1352628081,http://www.lawtime.cn/ask/question_9878938.html


In [51]:
d['type_1'] = d['fullURL'] #复制一列
d[0:10]

/root/anaconda3/envs/DataMining/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,a,b,realIP,fullURL,type_1
0,10000,12274,707080375,http://www.lawtime.cn/ask/question_2809224.html,http://www.lawtime.cn/ask/question_2809224.html
1,10001,12275,4195674807,http://www.lawtime.cn/ask/question_1009369.html,http://www.lawtime.cn/ask/question_1009369.html
2,10002,12276,2020773949,http://www.lawtime.cn/faguizt/23.html,http://www.lawtime.cn/faguizt/23.html
3,10003,12277,2020773949,http://www.lawtime.cn/faguizt/23.html,http://www.lawtime.cn/faguizt/23.html
4,10004,12278,1352628081,http://www.lawtime.cn/ask/question_9878938.html,http://www.lawtime.cn/ask/question_9878938.html
5,10005,12280,1685898872,http://www.lawtime.cn/ask/question_7821894.html,http://www.lawtime.cn/ask/question_7821894.html
6,10006,12281,3812410744,http://www.lawtime.cn/ask/question_10241534.html,http://www.lawtime.cn/ask/question_10241534.html
7,10007,12283,3000964314,http://www.lawtime.cn/ask/question_6489245.html,http://www.lawtime.cn/ask/question_6489245.html
8,10008,12284,2045253236,http://www.lawtime.cn/ask/question_4924778.html,http://www.lawtime.cn/ask/question_4924778.html
9,10009,12285,4120836154,http://www.lawtime.cn/askzt/zt_342.html,http://www.lawtime.cn/askzt/zt_342.html


In [52]:
d['type_1'][d['fullURL'].str.contains('(ask)|(askzt)')] = '咨询'
# d['type_1'][d['fullURL'].str.contains('(fagui)')] = '法律知识' 
# d['type_1'][d['fullURL'].str.contains('(hunyin)')] = '婚姻'
# d['type_1'][d['fullURL'].str.contains('(laodong)')] = '劳动'
# d['type_1'][d['fullURL'].str.contains('(xingfa)')] = '刑法' 
# d['type_1'][d['fullURL'].str.contains('(fangdichan)')] = '房地产'
# d['type_1'][d['fullURL'].str.contains('(zhaiquan)')] = '债券'
# d['type_1'][d['fullURL'].str.contains('(shuifa)')] = '税法'
# d['type_1'][d['fullURL'].str.contains('(minshi)')] = '民事'
# d['type_1'][d['fullURL'].str.contains('(paimai)')] = '拍卖'
# d['type_1'][d['fullURL'].str.contains('(hetong)')] = '合同'
# d['type_1'][d['fullURL'].str.contains('(lifa)')] = '立法'
# d['type_1'][d['fullURL'].str.contains('(jiaotong)')] = '交通'
# d['type_1'][d['fullURL'].str.contains('(xiaofei)')] = '消费'
# d['type_1'][d['fullURL'].str.contains('(gongsi)')] = '公司'
# d['type_1'][d['fullURL'].str.contains('(hehuo)')] = '合伙'

d['type_1'][d['fullURL'].str.contains('(fagui)')] = '咨询' 
d['type_1'][d['fullURL'].str.contains('(hunyin)')] = '咨询'
d['type_1'][d['fullURL'].str.contains('(laodong)')] = '咨询'
d['type_1'][d['fullURL'].str.contains('(xingfa)')] = '咨询' 
d['type_1'][d['fullURL'].str.contains('(fangdichan)')] = '咨询'
d['type_1'][d['fullURL'].str.contains('(zhaiquan)')] = '其他'
d['type_1'][d['fullURL'].str.contains('(shuifa)')] = '其他'
d['type_1'][d['fullURL'].str.contains('(minshi)')] = '其他'
d['type_1'][d['fullURL'].str.contains('(paimai)')] = '其他'
d['type_1'][d['fullURL'].str.contains('(hetong)')] = '其他'
d['type_1'][d['fullURL'].str.contains('(lifa)')] = '其他'
d['type_1'][d['fullURL'].str.contains('(jiaotong)')] = '其他'
d['type_1'][d['fullURL'].str.contains('(xiaofei)')] = '其他'
d['type_1'][d['fullURL'].str.contains('(gongsi)')] = '其他'
d['type_1'][d['fullURL'].str.contains('(hehuo)')] = '其他'

d[0:40]

/root/anaconda3/envs/DataMining/lib/python3.7/site-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
/root/anaconda3/envs/DataMining/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/root/anaconda3/envs/DataMining/lib/python3.7/site-packages/pandas/core/generic.py:9114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/root/anaconda3/envs/DataMining/lib/python3.7/site-packages/IPython/

,a,b,realIP,fullURL,type_1
0,10000,12274,707080375,http://www.lawtime.cn/ask/question_2809224.html,咨询
1,10001,12275,4195674807,http://www.lawtime.cn/ask/question_1009369.html,咨询
2,10002,12276,2020773949,http://www.lawtime.cn/faguizt/23.html,咨询
3,10003,12277,2020773949,http://www.lawtime.cn/faguizt/23.html,咨询
4,10004,12278,1352628081,http://www.lawtime.cn/ask/question_9878938.html,咨询
5,10005,12280,1685898872,http://www.lawtime.cn/ask/question_7821894.html,咨询
6,10006,12281,3812410744,http://www.lawtime.cn/ask/question_10241534.html,咨询
7,10007,12283,3000964314,http://www.lawtime.cn/ask/question_6489245.html,咨询
8,10008,12284,2045253236,http://www.lawtime.cn/ask/question_4924778.html,咨询
9,10009,12285,4120836154,http://www.lawtime.cn/askzt/zt_342.html,咨询


In [53]:
d['type_1'][d['type_1'].str.contains('(http)')] = '其他'
d[0:40]

/root/anaconda3/envs/DataMining/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,a,b,realIP,fullURL,type_1
0,10000,12274,707080375,http://www.lawtime.cn/ask/question_2809224.html,咨询
1,10001,12275,4195674807,http://www.lawtime.cn/ask/question_1009369.html,咨询
2,10002,12276,2020773949,http://www.lawtime.cn/faguizt/23.html,咨询
3,10003,12277,2020773949,http://www.lawtime.cn/faguizt/23.html,咨询
4,10004,12278,1352628081,http://www.lawtime.cn/ask/question_9878938.html,咨询
5,10005,12280,1685898872,http://www.lawtime.cn/ask/question_7821894.html,咨询
6,10006,12281,3812410744,http://www.lawtime.cn/ask/question_10241534.html,咨询
7,10007,12283,3000964314,http://www.lawtime.cn/ask/question_6489245.html,咨询
8,10008,12284,2045253236,http://www.lawtime.cn/ask/question_4924778.html,咨询
9,10009,12285,4120836154,http://www.lawtime.cn/askzt/zt_342.html,咨询


In [54]:
del d['a']
del d['b']

In [55]:
d[-5:]

,realIP,fullURL,type_1
995,2609113527,http://www.lawtime.cn/info/xiaofeizhe/wqzs/201...,其他
996,1754340622,http://www.lawtime.cn/faguizt/63.html,咨询
997,1754340622,http://www.lawtime.cn/faguizt/63.html,咨询
998,1642600052,http://www.lawtime.cn/info/gongsi/gudonghuijue...,其他
999,745280526,http://www.lawtime.cn/ask/question_7221496.html,咨询


In [56]:
types = d['type_1'].unique().tolist()
types

['咨询', '其他']

In [57]:
obj_mat = pd.DataFrame(columns=types)
obj_mat

,咨询,其他


In [58]:
import numpy as np
for index, row in d.iterrows():
    realIP = row['realIP']
    obj_mat.loc[realIP] = np.zeros(len(types),dtype=int)
obj_mat

,咨询,其他
707080375,0,0
4195674807,0,0
2020773949,0,0
1352628081,0,0
1685898872,0,0
...,...,...
254699722,0,0
2164003342,0,0
1754340622,0,0
1642600052,0,0


### 物品相似矩阵

In [59]:
def set_value(type_, types, v):
    for index, t in enumerate(types):
        if type_ == t:
            v[index] = 1
    print( np.sum(v))
    return v
for index, row in d.iterrows():
    realIP = row['realIP']
    type_ = row['type_1']
#     print(realIP)
    value = set_value(type_, types, obj_mat.loc[realIP].tolist() )
    
    obj_mat.loc[realIP] = value
#     breakabs
#     obj_mat.iloc[]
obj_mat

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
2
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
1
2
2
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
2
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
2
2
1
1
1
1
2
1
2
2
2
1
1
1
1
1


,咨询,其他
707080375,1,0
4195674807,1,0
2020773949,1,0
1352628081,1,0
1685898872,1,0
...,...,...
254699722,1,0
2164003342,1,0
1754340622,1,0
1642600052,0,1


# 推荐系统

## 基于物品的协同过滤算法

In [95]:
import numpy as np


class Recommender():
  
  sim = None #œ‡À∆∂»æÿ’Û
  
  def similarity(self, x, distance): #º∆À„œ‡À∆∂»æÿ’Ûµƒ∫Ø ˝
    y = np.ones((len(x), len(x)))
    for i in range(len(x)):
      for j in range(len(x)):
        y[i,j] = distance(x[i], x[j])
    return y
  
  def fit(self, x, distance = Jaccard): #—µ¡∑∫Ø ˝
    self.sim = self.similarity(x, distance)
  
  def recommend(self, a): #Õ∆ºˆ∫Ø ˝
    return np.dot(self.sim, a)*(1-a)



In [1]:
# 定义相似计算函数
from numpy import linalg as la

def ecludSim(a, b):
    return 1.0/(1.0 + la.norm(a - b))
def pearsSim(a, b):
    if len(a) < 3: 1.0
    return 0.5+0.5*corrcoef(a, b, rowvar = 0)[0][1]
def consSim(a,b):
    num = float(a.T * b)
    denom = la.norm(a) * la.norm(b)
    return 0.5+0.5*(num/denom)
def Jaccard(a, b): 
    return 1.0*(a*b).sum()/(a+b-a*b).sum()


In [62]:
obj_mat_ = obj_mat.T.as_matrix()

/root/anaconda3/envs/DataMining/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [63]:
a = obj_mat_[0]
b = obj_mat_[1]
1.0*(a*b).sum()/(a+b-a*b).sum()

0.0218408736349454

In [64]:
b

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,

In [65]:
r = Recommender()
r.fit(obj_mat_)
r.sim

array([[1.        , 0.02184087],
       [0.02184087, 1.        ]])

# 虚拟数据

In [100]:
def AP(ranked_list, ground_truth):
    """Compute the average precision (AP) of a list of ranked items
    """
    hits = 0
    sum_precs = 0
    for n in range(len(ranked_list)):
        if ranked_list[n] in ground_truth:
            hits += 1
            sum_precs += hits / (n + 1.0)
    if hits > 0:
        return sum_precs / len(ground_truth)
    else:
        return 0

In [87]:
data = {
    'User1': [3, 1, 2, 3, 3],
    'User2': [4, 3, 4, 3, 8],
    'User3': [3, 3, 1, 5, 4],
    'User4': [1, 5, 5, 2, 1]
}
df = pd.DataFrame(data).T
df.rename(columns={0:'itme1',
                   1:'itme2',
                   2:'itme2',
                   3:'itme3',
                   4:'itme4'},inplace=True)
df

,itme1,itme2,itme2,itme3,itme4
User1,3,1,2,3,3
User2,4,3,4,3,8
User3,3,3,1,5,4
User4,1,5,5,2,1


In [105]:

r = Recommender()
r.fit(df.T.as_matrix(), Jaccard)
r.sim

/root/anaconda3/envs/DataMining/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


array([[-2.69230769, -4.83333333, -4.28571429, -2.71428571, -2.        ],
       [-4.83333333, -2.2       , -2.33333333, -3.08333333, -2.75      ],
       [-4.28571429, -2.33333333, -2.09090909, -4.125     , -2.47368421],
       [-2.71428571, -3.08333333, -4.125     , -2.23809524, -2.11538462],
       [-2.        , -2.75      , -2.47368421, -2.11538462, -1.55172414]])

In [63]:
import pandas as pd
import numpy as np
df = pd.DataFrame(data = [3, 2, 3, 0, 1, 2], columns=["reli"])
# "log2(i+1)", "reli /log2(i+1)"
df = df.reset_index()
df = df.rename(columns = {"index":"i"})
df["i"] += 1
df["log2(i+1)"] = np.log2(df["i"]+1)
df["reli /log2(i+1)"] = df["reli"] /df["log2(i+1)"]
df

,i,reli,log2(i+1),reli /log2(i+1)
0,1,3,1.000000,3.000000
1,2,2,1.584963,1.261860
2,3,3,2.000000,1.500000
3,4,0,2.321928,0.000000
4,5,1,2.584963,0.386853
5,6,2,2.807355,0.712414


In [64]:
CG = df["reli"].sum()
CG

11

In [65]:
DCG = df["reli /log2(i+1)"].sum()
DCG

6.861126688593502

,i,reli,log2(i+1),reli /log2(i+1)
0,1,3,1.000000,3.000000
1,3,3,2.000000,1.500000
2,2,2,1.584963,1.261860
3,6,2,2.807355,0.712414
4,5,1,2.584963,0.386853
5,4,0,2.321928,0.000000


In [71]:
#理想情况排序
df["reli"] = df.sort_values(by="reli" , ascending=False).reset_index(drop=True)["reli"]
df["log2(i+1)"] = np.log2(df["i"]+1)
df["reli /log2(i+1)"] = df["reli"] /df["log2(i+1)"]
df

,i,reli,log2(i+1),reli /log2(i+1)
0,1,3,1.000000,3.000000
1,2,3,1.584963,1.892789
2,3,2,2.000000,1.000000
3,4,2,2.321928,0.861353
4,5,1,2.584963,0.386853
5,6,0,2.807355,0.000000


In [72]:
IDCG = df["reli /log2(i+1)"].sum()
IDCG

7.1409951840957

In [74]:
NDCG = DCG/IDCG
NDCG

0.9608081943360617

# 文档

请对比在推荐的准确度评价指标中的如下几个指标：

AP，MAP，AUC，NDCG。

请分别说明它们的含义，并举例说明它们在推荐系统中通常如何使用。必要时可以列出参考文献。请以PDF文件格式提交，文件名命名为学号+姓名。

# AP(Average Precision)
平均准确率AP，假使当我们使用google搜索某个关键词，返回了10个结果。当然最好的情况是这10个结果都是我们想要的相关信息。但是假如只有部分是相关的，比如5个，那么这5个结果如果被显示的比较靠前也是一个相对不错的结果。但是如果这个5个相关信息从第6个返回结果才开始出现，那么这种情况便是比较差的。这便是AP所反映的指标，与recall的概念有些类似，不过是“顺序敏感的recall。

$$AP_u = \frac{1}{\Omega_u}\sum\limits_{i\in\Omega_u}\frac{\sum_{j\in\Omega_u}h(p_{uj} < p_{ui})+1}{p_{ui}} $$
其中，$\Omega_u$表示ground-truth的结果，$p_{uj}$ 表示$i$物品在推荐列表中的位置，$p_{uj} < p_{ui}$ 表示$j$物品在推荐列表中排在$i$物品之前。

```py
def AP(ranked_list, ground_truth):
    """Compute the average precision (AP) of a list of ranked items
    """
    hits = 0
    sum_precs = 0
    for n in range(len(ranked_list)):
        if ranked_list[n] in ground_truth:
            hits += 1
            sum_precs += hits / (n + 1.0)
    if hits > 0:
        return sum_precs / len(ground_truth)
    else:
        return 0
```
# MAP(Mean Average Precision)
MAP表示所有用户的AP再取均值，计算公式如下：
$$ MAP = \frac{\sum_{u\in U}AP_u}{\left | U \right |}$$
```py
def MAP(ranked_list_s, ground_truth_s):
	sum_ap = 0
	for i in range(len(ranked_list_s)):
		sum_ap += AP(ranked_list_s[i], ground_truth_s[i])
	return sum_ap / len(ranked_list_s)
```
# AUC(Area Under Curve)
## ROC(Receiver operating characteristic)
ROC曲线的横轴为假正例率FPR(越小越好)，纵轴为真正例率TPR(越大越好)。
$$FPR = \frac{FP}{TN+FP}$$​	
$$TPR = \frac{TP}{TP+TN}$$

![在这里插入图片描述](https://img-blog.csdnimg.cn/20200326104926403.gif)
ROC计算过程：
 1. 首先每个样本都需要有一个label值，并且还需要一个预测的score值（取值0到1）
 2. 然后按这个score对样本由大到小进行排序，假设这些数据位于表格中的一列，从上到下依次降序
 3. 现在从上到下按照样本点的取值进行划分，位于分界点上面的我们把它归为预测为正样本，位于分界点下面的归为负样本
 4. 分别计算出此时的TPR和FPR，然后在图中绘制（FPR, TPR）点
 
 ![在这里插入图片描述](https://img-blog.csdnimg.cn/20200326104949424.png)
## AUC
AUC的含义：测试任意给一个正类样本和一个负类样本，正类样本的score有多大的概率大于负类样本的score。或者，任意给定一个负样本，所有正样本的score中有多大比例是大于该负类样本的score

```py
from sklearn.metrics import roc_curve
from sklearn import metrics
fpr, tpr, thresholds = roc_curve(
    test[:, 3], 
    predict_result,
    pos_label=1
)
auc = metrics.auc(fpr, tpr)

plt.plot(fpr, tpr, linewidth=2, label='ROC(area = %0.2f)' % auc, color='green')
plt.xlabel('Fail Positive Rate')
plt.ylabel('Ture Positive Rate')
plt.xlim(0, 1.05)
plt.ylim(0, 1.05)
plt.legend(loc=4)
plt.show()
```
![在这里插入图片描述](https://img-blog.csdnimg.cn/20200326105618888.png)
# NDCG(Normalized Discounted Cummulative Gain)
## CG(Cumulative Gain)
累积增益CG，推荐系统中CG表示将每个推荐结果相关性的分值累加后作为整个推荐列表的得分：
$$CG_k = \sum_{i=1} ^{k}rel_i$$
其中，$irel_i$表示位置iii的推荐结果的相关性，$k$表示推荐列表的大小。

CG没有考虑每个推荐结果处于不同位置对整个推荐结果的影响，例如，我们总是希望相关性大大的结果排在前面，相关性低的排在前面会影响用户体验。

## DCG(Discounted Cumulative Gain)
DCG在CG的基础上引入了位置影响因素，计算公式如下：
$$DCG_k = \sum_{i=1}^{k} \frac{2^{rel_i}-1} {log_2(i+1)}$$
从上面的式子可以得出：
1. 推荐结果的相关性越大，DCG越大。
2. 相关性好的排在推荐列表前面的话，推荐效果越好，DCG越大。

## NDCG(Normalized DCG)
DCG针对不同的推荐列表之间很难进行横向评估，而我们评估一个推荐系统不可能仅使用一个用户的推荐列表及相应结果进行评估，而是对整个测试集中的用户及其推荐列表结果进行评估。那么，不同用户的推荐列表的评估分数就需要进行归一化，也就是NDCG。

IDCG表示推荐系统某一用户返回的最好推荐结果列表， 即假设返回结果按照相关性排序， 最相关的结果放在最前面， 此序列的DCG为IDCG。因此DCG的值介于$(0,IDCG]$ ，故NDCG的值介于$(0,1]$，那么用户$u$的$NDCG@K$定义为：
$$CG_u@k = \frac{DCG_u@k}{IDCG_u}$$

平均NDCG的值为：
$$NDCG@k = \frac{\sum_{u \in U}NDCG_u@k}{IDCG_u}$$

参考 https://blog.csdn.net/qq_40006058/article/details/89432773